<a href="https://colab.research.google.com/github/Pallav-Lama/deeplearning-projects/blob/main/Hyperparameter_tuning_in_deep_leanring_using_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 36.6 MB/s 


In [ ]:
import tensorflow as tf

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
X_train = X_train/255.0 # rescaling
X_test = X_test/255.0

In [ ]:
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape = (28, 28)))

    hp_layer_1 = hp.Int('layer_1', min_value = 1, max_value = 1000, step = 100)
    hp_layer_2 = hp.Int('layer_2', min_value = 1, max_value = 1000, step = 100)
    hp_activation = hp.Choice('activation', values = ['relu', 'tanh'])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(tf.keras.layers.Dense(units = hp_layer_1, activation = hp_activation))
    model.add(tf.keras.layers.Dense(units = hp_layer_2, activation = hp_activation))

    model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

    return model

In [ ]:
import keras_tuner as kt

In [ ]:
tuner = kt.Hyperband(build_model,
             objective = 'val_accuracy',
             max_epochs= 10,
             factor = 3,
             directory = 'kt',
             project_name = 'x'
             )

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
tuner.search(X_train,
             y_train,
             epochs=10,
             validation_split=0.2,
             callbacks=[stop_early]) # search optimal model in train space

Trial 30 Complete [00h 00m 44s]
val_accuracy: 0.9605833292007446

Best val_accuracy So Far: 0.9778333306312561
Total elapsed time: 00h 11m 14s


In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0] # get top model

In [ ]:
best_hps

In [ ]:
best_hps.values

{'layer_1': 501,
 'layer_2': 901,
 'activation': 'relu',
 'learning_rate': 0.0001,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [ ]:
best_hps.__dict__

{'_name_scopes': [],
 '_conditions': [],
 '_hps': defaultdict(list,
             {'layer_1': [Int(name: "layer_1", min_value: 1, max_value: 1000, step: 100, sampling: None, default: 1)],
              'layer_2': [Int(name: "layer_2", min_value: 1, max_value: 1000, step: 100, sampling: None, default: 1)],
              'activation': [Choice(name: "activation", values: ['relu', 'tanh'], ordered: False, default: relu)],
              'learning_rate': [Choice(name: "learning_rate", values: [0.01, 0.001, 0.0001], ordered: True, default: 0.01)]}),
 '_space': [Int(name: "layer_1", min_value: 1, max_value: 1000, step: 100, sampling: None, default: 1),
  Int(name: "layer_2", min_value: 1, max_value: 1000, step: 100, sampling: None, default: 1),
  Choice(name: "activation", values: ['relu', 'tanh'], ordered: False, default: relu),
  Choice(name: "learning_rate", values: [0.01, 0.001, 0.0001], ordered: True, default: 0.01)],
 'values': {'layer_1': 501,
  'layer_2': 901,
  'activation': 'relu',
  

In [ ]:
best_hps['layer_1']

501

In [ ]:
best_hps['layer_2']

901

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=20, validation_split=0.2,
                    callbacks=[stop_early])

Epoch 1/20
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3623 - accuracy: 0.9031 - val_loss: 0.1713 - val_accuracy: 0.9522
Epoch 2/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1483 - accuracy: 0.9571 - val_loss: 0.1321 - val_accuracy: 0.9622
Epoch 3/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.1015 - accuracy: 0.9706 - val_loss: 0.1213 - val_accuracy: 0.9632
Epoch 4/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0752 - accuracy: 0.9789 - val_loss: 0.0913 - val_accuracy: 0.9722
Epoch 5/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0568 - accuracy: 0.9831 - val_loss: 0.0873 - val_accuracy: 0.9730
Epoch 6/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0430 - accuracy: 0.9877 - val_loss: 0.0828 - val_accuracy: 0.9737
Epoch 7/20
1500/1500 [==============================] - 4s 3ms/step - loss: 0.0331 - accuracy: 0.9908 - val_loss: 0.0820 - val_accuracy: